In [ ]:
import numpy as np
import os
from tensorflow.keras.applications import MobileNetV3Small, ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
import zipfile
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

In [ ]:
zip_path = '/content/drive/MyDrive/original+smote+image_processing.zip'

extracted_path = '/content/dataset/'

os.makedirs(extracted_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

extracted_folders = os.listdir(extracted_path)
print("Extracted folders:", extracted_folders)

for folder in extracted_folders:
    folder_path = os.path.join(extracted_path, folder)
    if os.path.isdir(folder_path):
        images_in_folder = os.listdir(folder_path)
        print(f"Images in '{folder}':", images_in_folder)


In [ ]:
train_dir = 'dataset/split/train'
test_dir = 'dataset/split/test'

In [ ]:
BATCH_SIZE=256 # 32 for Original Dataset, 128 for Smote Dataset, 256 for Augmented Dataset

In [ ]:
train_data = image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

In [ ]:
model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in model.layers:
	layer.trainable = False
model.summary()

In [ ]:
train_features = []
train_labels = []

for batch_images, batch_labels in train_data:
    features_batch = model.predict(batch_images)
    train_features.append(features_batch)
    train_labels.append(batch_labels)

train_features = np.concatenate(train_features)
train_labels = np.concatenate(train_labels)
train_features = train_features.reshape((train_features.shape[0], -1))

In [ ]:
test_features = []
test_labels = []

for batch_images, batch_labels in test_data:
    features_batch = model.predict(batch_images)
    test_features.append(features_batch)
    test_labels.append(batch_labels)

test_features = np.concatenate(test_features)
test_labels = np.concatenate(test_labels)
test_features = test_features.reshape((test_features.shape[0], -1))

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

In [ ]:
rf_classifier.fit(train_features, np.argmax(train_labels, axis=1))

In [ ]:
train_predictions_rf = rf_classifier.predict(train_features)
test_predictions_rf = rf_classifier.predict(test_features)

In [ ]:
svm_classifier.fit(train_features, np.argmax(train_labels, axis=1))

In [ ]:
train_predictions_svm = svm_classifier.predict(train_features)
test_predictions_svm = svm_classifier.predict(test_features)

In [ ]:
print (f"Training accuracy (RF) =  {accuracy_score(np.argmax(train_labels, axis=1), train_predictions_rf)}")
print (f"Testing accuracy (RF) =  {accuracy_score(np.argmax(test_labels, axis=1), test_predictions_rf)}")
print(f"Training accuracy (SVM) = {accuracy_score(np.argmax(train_labels, axis=1), train_predictions_svm)}")
print(f"Testing accuracy (SVM) = {accuracy_score(np.argmax(test_labels, axis=1), test_predictions_svm)}")